In [1]:
import sys
sys.path.append("..")


from experiments.plt_pab_fae import plt_fab_fae
from protocol.BB84EveTrainableProtocol import BB84EveTrainableProtocol
from protocol.BB84TrainableProtocol import BB84TrainableProtocol
from protocol.BB84Protocol import BB84Protocol
from protocol.connection_elements.MultiQubitQCLEve import MultiQubitQCLEve
from protocol.connection_elements.error_correction.ThreeQubitCorrection import ThreeQubitBitFlipEncoder, ThreeQubitBitFlipDecoder



ImportError: libcustatevec.so.1: cannot open shared object file: No such file or directory

In [ ]:
layers = [ThreeQubitBitFlipEncoder(), ThreeQubitBitFlipDecoder()]
eve = MultiQubitQCLEve()

eve_protocol = BB84EveTrainableProtocol(n_bits=512, elements=[layers[0], eve, layers[1]], channel_size=3,
                                        seed=0, alpha=1, learning_rate=0.05, torch_device="cuda", backend_device="GPU")

qc, _ = eve_protocol.qc_with_ctx()
qc.draw("mpl", expr_len=3)

In [ ]:
def train_eve():
    bob_qber, eve_qber = [], []
    qber = eve_protocol.run()
    bob_qber.append(qber['bob_qber'])
    eve_qber.append(qber['eve_qber'])
    print(f'start QBER: {qber}')
    for epoch in range(50):
        loss = eve_protocol.train()
        qber = eve_protocol.run()
        bob_qber.append(qber['bob_qber'])
        eve_qber.append(qber['eve_qber'])
        if epoch % 5 == 0 or epoch == 49:
            print(f'epoch: {epoch}, loss: {loss}, QBER: {qber}')
    return bob_qber, eve_qber

In [ ]:
protocol = BB84Protocol(512, layers, channel_size=3, seed=0)
print(protocol.run())

protocol.qc_with_ctx()[0].decompose().draw("mpl")

# Krok 2 - trenowanie tylko Eve, warstwy Alice i Boba zamrożone
Uczymy samą Eve, docelowa dokładność Boba to 0.8. Startujemy od parametrów Alice i Boba wytrenowanych w poprzednim kroku i Eve jako identyczność.

In [ ]:
params = eve_protocol.get_all_parameters()
for p in eve.trainable_parameters():
    k = p.name
    params[k] = 0

eve_protocol.set_parameters(params)


In [ ]:
eve_protocol.f_value = 0.8
b_q, e_q = train_eve()

In [ ]:
plt_fab_fae(b_q, e_q, 0.8)

In [ ]:
print("All parameters:", eve_protocol.get_all_parameters())

In [ ]:
eve_protocol._qc.assign_parameters(eve_protocol.get_all_parameters()).draw("mpl")